<a href="https://colab.research.google.com/github/woo13sd/rupture-theory/blob/main/notebooks/Rupture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Install all required packages and create project directories

# System packages
!apt-get update -qq
!apt-get install -y gfortran wget unzip

# Python packages
!pip install numpy matplotlib scipy h5py cython --quiet

# Create only the necessary directories
!mkdir -p /content/data
!mkdir -p /content/images
!mkdir -p /content/camb_utils

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gfortran is already the newest version (4:11.2.0-1ubuntu1).
unzip is already the newest version (6.0-26ubuntu3.2).
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


In [2]:
# Cell 2: Clone CAMB from GitHub (main branch)
!git clone --depth=1 https://github.com/cmbant/CAMB.git /content/CAMB

# List to confirm
!ls /content/CAMB

Cloning into '/content/CAMB'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 147 (delta 8), reused 63 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 13.35 MiB | 7.29 MiB/s, done.
Resolving deltas: 100% (8/8), done.
camb		 fortran   LICENCE.txt	   README.rst
CONTRIBUTING.md  forutils  MANIFEST.in	   requirements.txt
docs		 inifiles  pyproject.toml  setup.py


In [4]:
# Cell 3: Copy your uploaded custom .f90 files into the CAMB fortran directory (correct case)
import shutil
import os

source_dir = "/content/camb_utils"
target_dir = "/content/CAMB/fortran"

# Updated file names with correct casing
f90_files = [
    "InitialPower.f90",
    "reionization.f90"
]

for file in f90_files:
    src = os.path.join(source_dir, file)
    dst = os.path.join(target_dir, file)
    shutil.copyfile(src, dst)

print("✅ Modified .f90 files copied to CAMB/fortran/")

✅ Modified .f90 files copied to CAMB/fortran/


In [5]:
# Cell 4: Copy custom .ini files to CAMB/inifiles/
import shutil
import os

ini_files = [
    "rt_f1e08.ini",
    "rt_f1e05.ini",
    "rt_f1e02.ini"
]

source_dir = "/content/camb_utils"
target_dir = "/content/CAMB/inifiles"

for file in ini_files:
    src = os.path.join(source_dir, file)
    dst = os.path.join(target_dir, file)
    shutil.copyfile(src, dst)

print("✅ Custom .ini files copied to CAMB/inifiles/")

✅ Custom .ini files copied to CAMB/inifiles/


In [6]:
# Cell 5a: Compile constants.f90 manually
import subprocess

build_dir = "/content/CAMB/fortran"
filename = "constants.f90"

# Compile
compile_cmd = [
    "gfortran",
    "-c",                               # Compile only (no linking)
    "-J", build_dir,                    # Output .mod files here
    "-o", f"{build_dir}/constants.o",  # Output .o file
    f"{build_dir}/{filename}",         # Input source file
]

# Run command
result = subprocess.run(compile_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Show result
if result.returncode == 0:
    print(f"✅ {filename} compiled successfully.")
else:
    print(f"❌ Error compiling {filename}:\n{result.stderr}")

✅ constants.f90 compiled successfully.


In [7]:
# Cell 6: Compile MiscUtils.f90 with proper lowercase handling for the .mod file

import subprocess
import shutil
import os

miscutils_src = "/content/forutils/MiscUtils.f90"
mod_file = "miscutils.mod"
obj_file = "MiscUtils.o"

print("🔧 Cell 6: Compiling MiscUtils.f90...")

try:
    subprocess.run(["gfortran", "-c", miscutils_src], check=True)

    if os.path.exists(obj_file) and os.path.exists(mod_file):
        shutil.move(obj_file, "/content/CAMB/fortran/")
        shutil.move(mod_file, "/content/CAMB/fortran/")
        print("✅ Cell 6: Compilation successful. Files moved.")
    else:
        print("❌ Cell 6: Compilation did not produce expected output files.")
except subprocess.CalledProcessError as e:
    print("❌ Cell 6: Compilation error.")
    print(e)

🔧 Cell 6: Compiling MiscUtils.f90...
❌ Cell 6: Compilation error.
Command '['gfortran', '-c', '/content/forutils/MiscUtils.f90']' returned non-zero exit status 1.


In [8]:
# Cell 7: Download forutils and verify FileUtils.f90 is available

import os
import subprocess

print("🌐 Cloning CAMB forutils repo...")

# Clean up if already exists (optional, comment out if not needed)
if os.path.exists("/content/CAMB/forutils"):
    subprocess.run(["rm", "-rf", "/content/CAMB/forutils"])

# Clone the repository
subprocess.run(["git", "clone", "https://github.com/cmbant/forutils.git", "/content/CAMB/forutils"])

# Check required file
required = "FileUtils.f90"
file_path = f"/content/CAMB/forutils/{required}"

if os.path.exists(file_path):
    print(f"✅ {required} found at {file_path}")
else:
    raise FileNotFoundError(f"❌ {required} not found in /content/CAMB/forutils/")

🌐 Cloning CAMB forutils repo...
✅ FileUtils.f90 found at /content/CAMB/forutils/FileUtils.f90


In [17]:
# 🛠 Cell 8: Patch config.f90 to remove MpiUtils reference

config_path = "/content/CAMB/fortran/config.f90"

with open(config_path, "r") as f:
    lines = f.readlines()

patched_lines = []
for line in lines:
    if "use MpiUtils" in line:
        patched_lines.append("! use MpiUtils  ! 🔧 MPI disabled in Colab\n")
    else:
        patched_lines.append(line)

with open(config_path, "w") as f:
    f.writelines(patched_lines)

print("✅ Cell 8: Patched config.f90 to bypass MPI dependency.")

✅ Cell 8: Patched config.f90 to bypass MPI dependency.
